## Carregando os dados e checando se está ok

In [2]:
from pathlib import Path
import pandas as pd
csv_path = Path(r"C:\Users\idolo\OneDrive\Área de Trabalho\Curso - Modelos Computacionais\Projeto") / "Dataset_Modelos_Computacionais_clean.csv"

df = pd.read_csv(csv_path, encoding="utf-8-sig")


df.head()


,Ano,Mes,Dengue,Zika,Chik,LeishVis,LeishTeg,LeishT,Precipt,AvgTemp,MaxTemp,MinTemp,AvgHumid,AvgWin,Season,Season2
0,2014,Janeiro,2901,NaN,NaN,15,31,46,138.90,25.12,25.86,24.41,64.82,1.56,Summer,Wet
1,2014,Fevereiro,10196,NaN,NaN,20,32,52,139.67,25.79,26.49,25.10,58.32,1.74,Summer,Wet
2,2014,Marco,29647,NaN,NaN,18,29,47,150.00,23.86,24.47,23.23,70.44,1.74,Summer,Wet
3,2014,Abril,85767,NaN,NaN,20,22,42,103.92,22.12,22.69,21.59,74.90,1.49,Fall,Dry
4,2014,Maio,62901,NaN,NaN,15,16,31,57.20,19.72,20.28,19.18,70.97,1.44,Fall,Dry


##Medidas descritivas (geral)

In [3]:
df.describe()

,Ano,Dengue,Zika,Chik,LeishVis,LeishTeg,LeishT,Precipt,AvgTemp,MaxTemp,MinTemp,AvgHumid,AvgWin
count,132.000000,132.000000,108.000000,96.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000,132.000000
mean,2019.000000,37008.848485,215.425926,927.187500,10.909091,27.848485,38.757576,119.955227,22.179091,22.785227,21.577424,68.240758,1.579091
std,3.174324,88805.108924,414.422720,1432.025759,4.457526,8.631104,10.511441,80.096108,2.207175,2.269410,2.236895,6.332219,0.223120
min,2014.000000,310.000000,28.000000,96.000000,2.000000,7.000000,12.000000,4.970000,16.850000,16.750000,15.650000,50.140000,1.140000
25%,2016.000000,1681.500000,68.750000,265.500000,7.000000,22.000000,31.750000,50.580000,20.165000,20.742500,19.527500,64.165000,1.420000
50%,2019.000000,5134.500000,109.000000,387.000000,11.000000,27.000000,38.000000,111.070000,22.955000,23.560000,22.385000,68.915000,1.565000
75%,2022.000000,32834.250000,178.000000,875.250000,14.000000,33.250000,46.000000,168.655000,23.912500,24.530000,23.322500,73.152500,1.755000
max,2024.000000,619390.000000,2874.000000,8154.000000,22.000000,54.000000,66.000000,370.270000,25.960000,26.660000,25.290000,79.880000,2.070000


## df.info descreve as variáveis

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Ano       132 non-null    int64  
 1   Mes       132 non-null    object 
 2   Dengue    132 non-null    int64  
 3   Zika      108 non-null    float64
 4   Chik      96 non-null     float64
 5   LeishVis  132 non-null    int64  
 6   LeishTeg  132 non-null    int64  
 7   LeishT    132 non-null    int64  
 8   Precipt   132 non-null    float64
 9   AvgTemp   132 non-null    float64
 10  MaxTemp   132 non-null    float64
 11  MinTemp   132 non-null    float64
 12  AvgHumid  132 non-null    float64
 13  AvgWin    132 non-null    float64
 14  Season    132 non-null    object 
 15  Season2   132 non-null    object 
dtypes: float64(8), int64(5), object(3)
memory usage: 16.6+ KB


## Pré tratamento dos dados:
- Criar uma coluna "Data" para armazenar informação de mês e ano (não será uma variável preditora)
- Retirar ano;
- Retirar mês;
- Deixar apenas temperatura média
- Excluir variáveis preditoras colineares com VIF>10;
- Excluir outliers dos números de casos;

In [5]:
## Criando a coluna data
# -----------------------------------------------------------
# 2. Criar coluna de DATA a partir de Ano + Mes
# -----------------------------------------------------------
mes_map = {
    "Janeiro": 1,
    "Fevereiro": 2,
    "Marco": 3,
    "Março": 3,  # caso apareça com acento
    "Abril": 4,
    "Maio": 5,
    "Junho": 6,
    "Julho": 7,
    "Agosto": 8,
    "Setembro": 9,
    "Outubro": 10,
    "Novembro": 11,
    "Dezembro": 12
}

df["MesNum"] = df["Mes"].map(mes_map)

# cria coluna Data (vou usar dia 1 de cada mês)
df["Data"] = pd.to_datetime(dict(year=df["Ano"], month=df["MesNum"], day=1))

# definir Data como índice temporal e ordenar
df = df.set_index("Data").sort_index()


In [6]:
## Removendo ano, mês, temperatura máxima e mínima
df = df.drop(columns=["Ano", "Mes", "MesNum", "MinTemp", "MaxTemp"])


# Check the result
print(df.head())

            Dengue  Zika  Chik  LeishVis  LeishTeg  LeishT  Precipt  AvgTemp  \
Data                                                                           
2014-01-01    2901   NaN   NaN        15        31      46   138.90    25.12   
2014-02-01   10196   NaN   NaN        20        32      52   139.67    25.79   
2014-03-01   29647   NaN   NaN        18        29      47   150.00    23.86   
2014-04-01   85767   NaN   NaN        20        22      42   103.92    22.12   
2014-05-01   62901   NaN   NaN        15        16      31    57.20    19.72   

            AvgHumid  AvgWin  Season Season2  
Data                                          
2014-01-01     64.82    1.56  Summer     Wet  
2014-02-01     58.32    1.74  Summer     Wet  
2014-03-01     70.44    1.74  Summer     Wet  
2014-04-01     74.90    1.49    Fall     Dry  
2014-05-01     70.97    1.44    Fall     Dry  


In [14]:
## Carregando biblioteca para cálculo do VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [60]:
# -------------------------------------------------------------------
# 1. Carregar o dataset
# -------------------------------------------------------------------
csv_path = Path(r"C:\Users\idolo\OneDrive\Área de Trabalho\Curso - Modelos Computacionais\Projeto") / "Dataset_Modelos_Computacionais_clean.csv"

df = pd.read_csv(csv_path, encoding="utf-8-sig")

# -------------------------------------------------------------------
# 2. Remover colunas "Ano" e "Mes" (já decidido no seu projeto)
# -------------------------------------------------------------------
df = df.drop(columns=["Ano", "Mes"])

# -------------------------------------------------------------------
# 3. Definir quais variáveis vão entrar no cálculo do VIF
#    Aqui eu assumo que você quer modelar, por exemplo, DENGUE.
#    Então as preditoras são as variáveis climáticas numéricas.
# -------------------------------------------------------------------
# Se quiser mudar o alvo depois (ex: "Zika"), basta trocar aqui.
target = "Dengue"

# Lista de preditoras que vão para o VIF (climáticas)
preditoras_vif = ["Precipt", "AvgTemp", "AvgHumid", "AvgWin"]

# Criar matriz X só com essas colunas (sem valores faltantes)
X = df[preditoras_vif].dropna().reset_index(drop=True)

# -------------------------------------------------------------------
# 4. Calcular o VIF para cada preditora
# -------------------------------------------------------------------
vif_table = pd.DataFrame()
vif_table["variavel"] = preditoras_vif
vif_table["VIF"] = [
    variance_inflation_factor(X.values, i)
    for i in range(X.shape[1])
]

# Ordenar da maior colinearidade para a menor
vif_table = vif_table.sort_values(by="VIF", ascending=False).reset_index(drop=True)

print("Tabela de VIF (antes da exclusão):")
print(vif_table)

# -------------------------------------------------------------------
# 5. Identificar variáveis com VIF > 10 e removê-las
# -------------------------------------------------------------------
limite_vif = 10  # critério da sua anotação

vars_alto_vif = vif_table.loc[vif_table["VIF"] > limite_vif, "variavel"].tolist()

print("\nVariáveis com VIF > 10 (candidatas à exclusão):")
print(vars_alto_vif)

# Criar um novo DataFrame SEM essas variáveis colineares
df_sem_colinear = df.drop(columns=vars_alto_vif)

print("\nColunas finais em df_sem_colinear (após remover VIF > 10):")
print(df_sem_colinear.columns)

Tabela de VIF (antes da exclusão):
   variavel         VIF
0   AvgTemp  111.903118
1  AvgHumid   72.214836
2    AvgWin   50.060495
3   Precipt    4.322273

Variáveis com VIF > 10 (candidatas à exclusão):
['AvgTemp', 'AvgHumid', 'AvgWin']

Colunas finais em df_sem_colinear (após remover VIF > 10):
Index(['Dengue', 'Zika', 'Chik', 'LeishVis', 'LeishTeg', 'LeishT', 'Precipt',
       'MaxTemp', 'MinTemp', 'Season', 'Season2'],
      dtype='object')


## Retirar outliers

In [7]:
from pathlib import Path
import pandas as pd
import numpy as np


## Outliers - Método IQR e 99%

In [8]:
# =======================================
# 0) IMPORTS E CAMINHOS
# =======================================
from pathlib import Path
import pandas as pd

# Caminho da pasta do projeto (mesmo que você vem usando)
base_path = Path(r"C:\Users\idolo\OneDrive\Área de Trabalho\Curso - Modelos Computacionais\Projeto")
csv_path = base_path / "Dataset_Modelos_Computacionais_clean.csv"

# =======================================
# 1) LEITURA DO CSV E PRÉ-TRATAMENTO BÁSICO
#    - cria coluna Data
#    - usa Data como índice
#    - remove Ano, Mes, MesNum, MinTemp, MaxTemp
# =======================================

# Leitura
df = pd.read_csv(csv_path, encoding="utf-8-sig")

# (Opcional) olhar um pouco os dados
print("Primeiras linhas do df original:")
print(df.head(), "\n")

print("Resumo das colunas:")
print(df.info(), "\n")

# Map de meses em português -> número
mes_map = {
    "Janeiro": 1,
    "Fevereiro": 2,
    "Marco": 3,
    "Março": 3,  # caso apareça com acento
    "Abril": 4,
    "Maio": 5,
    "Junho": 6,
    "Julho": 7,
    "Agosto": 8,
    "Setembro": 9,
    "Outubro": 10,
    "Novembro": 11,
    "Dezembro": 12
}

# Criar MesNum e coluna Data
df["MesNum"] = df["Mes"].map(mes_map)
df["Data"] = pd.to_datetime(dict(year=df["Ano"], month=df["MesNum"], day=1))

# Usar Data como índice temporal
df = df.set_index("Data").sort_index()

# Remover colunas que NÃO queremos como preditoras
df = df.drop(columns=["Ano", "Mes", "MesNum", "MinTemp", "MaxTemp"], errors="ignore")

print("Colunas após pré-tratamento (df):")
print(df.columns, "\n")

# =======================================
# 2) OUTLIERS POR IQR (POR MÊS)
#    -> df_iqr_sem_outliers
#    + CSVs em pasta Outliers_RF
# =======================================

# Copia de trabalho
df_out = df.copy()
df_out.index = pd.to_datetime(df_out.index)

# Coluna explícita de Data para exportar
df_out["Data"] = df_out.index

# Mês do ano (1–12) e ano (pra análise IQR e CSV)
df_out["Mes_do_ano"] = df_out["Data"].dt.month
df_out["Ano"] = df_out["Data"].dt.year

# Lista de doenças
disease_cols = ["Dengue", "Zika", "Chik", "LeishVis", "LeishTeg", "LeishT"]

# Pasta para salvar os CSVs de outliers IQR
out_dir_iqr = base_path / "Outliers_RF"
out_dir_iqr.mkdir(exist_ok=True)

resumo_outliers = []

for col in disease_cols:
    if col not in df_out.columns:
        print(f"[AVISO] {col} não está no dataframe. Pulando.")
        continue

    print(f"\n=== Avaliando outliers (IQR por mês) para: {col} ===")

    sub = df_out[["Data", "Ano", "Mes_do_ano", col]].copy()
    sub = sub.dropna(subset=[col])

    if sub.empty:
        print(f"[AVISO] {col} só tem NaN. Pulando.")
        continue

    # Q1, Q3 e IQR por mês do ano
    grupo = sub.groupby("Mes_do_ano")[col]
    q1 = grupo.quantile(0.25)
    q3 = grupo.quantile(0.75)

    sub = sub.join(q1.rename("Q1"), on="Mes_do_ano")
    sub = sub.join(q3.rename("Q3"), on="Mes_do_ano")
    sub["IQR"] = sub["Q3"] - sub["Q1"]

    sub["Limite_inferior"] = sub["Q1"] - 1.5 * sub["IQR"]
    sub["Limite_superior"] = sub["Q3"] + 1.5 * sub["IQR"]

    sub["is_outlier"] = (sub[col] < sub["Limite_inferior"]) | (sub[col] > sub["Limite_superior"])

    outliers = sub[sub["is_outlier"]].copy()

    print(f"Total de observações para {col}: {len(sub)}")
    print(f"Número de outliers detectados: {len(outliers)}")

    resumo_outliers.append({
        "doenca": col,
        "n_total": len(sub),
        "n_outliers": len(outliers)
    })

    # salvar CSV só de outliers dessa doença
    if len(outliers) > 0:
        cols_salvar = [
            "Data", "Ano", "Mes_do_ano", col,
            "Q1", "Q3", "Limite_inferior", "Limite_superior", "is_outlier"
        ]
        out_csv_path = out_dir_iqr / f"outliers_{col}_IQR.csv"
        outliers[cols_salvar].to_csv(out_csv_path, index=False, encoding="utf-8-sig")
        print(f"Outliers de {col} salvos em: {out_csv_path}")
    else:
        print(f"Nenhum outlier detectado para {col} com esse critério.")

    # Escrever flag de outlier de volta no df_out (alinhando por Data)
    flag_series = sub.set_index("Data")["is_outlier"]
    df_out[f"{col}_outlier_IQR"] = df_out.index.isin(flag_series[flag_series].index)

# Resumo geral IQR
resumo_iqr_df = pd.DataFrame(resumo_outliers)
print("\nResumo geral de outliers por doença (IQR):")
print(resumo_iqr_df)

# Construir df_iqr_sem_outliers (remover qualquer linha com outlier em qualquer doença)
iqr_flag_cols = [c for c in df_out.columns if c.endswith("_outlier_IQR")]

if len(iqr_flag_cols) > 0:
    mask_any_iqr = df_out[iqr_flag_cols].any(axis=1)
    df_iqr_sem_outliers = df_out[~mask_any_iqr].copy()
else:
    df_iqr_sem_outliers = df_out.copy()

print("\nLinhas totais em df_out:", len(df_out))
print("Linhas após remover outliers (IQR por mês):", len(df_iqr_sem_outliers))

# Limpar colunas auxiliares do df_iqr_sem_outliers para modelagem
df_iqr_sem_outliers = df_iqr_sem_outliers.drop(
    columns=iqr_flag_cols + ["Mes_do_ano", "Ano", "Data"],
    errors="ignore"
)

print("\nColunas de df_iqr_sem_outliers:")
print(df_iqr_sem_outliers.columns, "\n")

# =======================================
# 3) OUTLIERS "EXTREMOS" POR PERCENTIL GLOBAL (p99)
#    -> df_p99_sem_extremos
#    + pasta Outliers_RF_p99
# =======================================

df_thresh = df.copy()
df_thresh.index = pd.to_datetime(df_thresh.index)

percentile_cutoff = 0.99  # 99º percentil
thresholds = []

for col in disease_cols:
    if col not in df_thresh.columns:
        print(f"[AVISO] {col} não está no dataframe. Pulando.")
        continue

    s = df_thresh[col].dropna()

    if s.empty:
        print(f"[AVISO] {col} só tem NaN. Pulando.")
        continue

    # X: valor acima do qual vamos considerar "surto extremo"
    X = s.quantile(percentile_cutoff)

    thresholds.append({
        "doenca": col,
        f"p{int(percentile_cutoff*100)}": X,
        "max": s.max()
    })

    # cria uma flag de "surto extremo" para essa doença
    df_thresh[f"{col}_extremo"] = df_thresh[col] > X

# tabela com cutoffs por doença
thresholds_df = pd.DataFrame(thresholds)
print("\nLimiares de 'surto extremo' por doença (p99):")
print(thresholds_df)

# construir df_sem_extremos removendo qualquer linha com *_extremo = True
extreme_flag_cols = [c for c in df_thresh.columns if c.endswith("_extremo")]

if len(extreme_flag_cols) > 0:
    mask_any_extreme = df_thresh[extreme_flag_cols].any(axis=1)
    df_sem_extremos = df_thresh[~mask_any_extreme].copy()
else:
    df_sem_extremos = df_thresh.copy()

print("\nLinhas totais em df_thresh:", len(df_thresh))
print("Linhas após remover surtos extremos (p99):", len(df_sem_extremos))

# CRIAR PASTA E SALVAR CSVs DO P99
out_dir_percentil = base_path / f"Outliers_RF_p{int(percentile_cutoff*100)}"
out_dir_percentil.mkdir(exist_ok=True)

# 1) salvar limiares por doença
thresholds_csv_path = out_dir_percentil / f"limiares_por_doenca_p{int(percentile_cutoff*100)}.csv"
thresholds_df.to_csv(thresholds_csv_path, index=False, encoding="utf-8-sig")
print(f"\nArquivo de limiares salvo em: {thresholds_csv_path}")

# 2) salvar dataset sem surtos extremos (incluindo flags *_extremo)
dataset_sem_extremos_path = out_dir_percentil / f"Dataset_sem_extremos_p{int(percentile_cutoff*100)}.csv"
df_sem_extremos.to_csv(dataset_sem_extremos_path, encoding="utf-8-sig", index=True)
print(f"Dataset sem surtos extremos salvo em: {dataset_sem_extremos_path}")

# 3) salvar CSV de outliers por doença
df_export = df_thresh.copy()
df_export["Data"] = df_export.index

for col in disease_cols:
    flag_col = f"{col}_extremo"
    if flag_col not in df_export.columns:
        continue

    outliers_col = df_export[df_export[flag_col]].copy()
    if outliers_col.empty:
        print(f"Nenhum surto extremo para {col} com p{int(percentile_cutoff*100)}.")
        continue

    cols_salvar = ["Data", col, flag_col]
    out_csv_path = out_dir_percentil / f"outliers_{col}_p{int(percentile_cutoff*100)}.csv"
    outliers_col[cols_salvar].to_csv(out_csv_path, index=False, encoding="utf-8-sig")
    print(f"Outliers de {col} salvos em: {out_csv_path}")

# 4) DataFrame final para usar nos modelos (sem flags *_extremo)
df_p99_sem_extremos = df_sem_extremos.drop(columns=extreme_flag_cols + ["Data"], errors="ignore")

print("\nShape de df_p99_sem_extremos (sem flags *_extremo):", df_p99_sem_extremos.shape)
print("Colunas de df_p99_sem_extremos:")
print(df_p99_sem_extremos.columns)


Primeiras linhas do df original:
    Ano        Mes  Dengue  Zika  Chik  LeishVis  LeishTeg  LeishT  Precipt  \
0  2014    Janeiro    2901   NaN   NaN        15        31      46   138.90   
1  2014  Fevereiro   10196   NaN   NaN        20        32      52   139.67   
2  2014      Marco   29647   NaN   NaN        18        29      47   150.00   
3  2014      Abril   85767   NaN   NaN        20        22      42   103.92   
4  2014       Maio   62901   NaN   NaN        15        16      31    57.20   

   AvgTemp  MaxTemp  MinTemp  AvgHumid  AvgWin  Season Season2  
0    25.12    25.86    24.41     64.82    1.56  Summer     Wet  
1    25.79    26.49    25.10     58.32    1.74  Summer     Wet  
2    23.86    24.47    23.23     70.44    1.74  Summer     Wet  
3    22.12    22.69    21.59     74.90    1.49    Fall     Dry  
4    19.72    20.28    19.18     70.97    1.44    Fall     Dry   

Resumo das colunas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data col

## Random Forest

In [9]:
# =====================================================
# IMPORTS E CAMINHO BASE
# =====================================================
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


## Random Forest - Todos os modelos - CORRETO

In [10]:

# =====================================================
# CONFIGURAÇÕES INICIAIS
# =====================================================
base_path = Path(r"C:\Users\idolo\OneDrive\Área de Trabalho\Curso - Modelos Computacionais\Projeto")

# Nome da pasta mãe que vai conter tudo dessa rodada
pasta_mae_nome = "Resultados_Gerais_NoSeason"

possible_disease_cols = ["Dengue", "Zika", "Chik", "LeishVis", "LeishTeg", "LeishT"]
climate_cols = ["Precipt", "AvgTemp", "MaxTemp", "MinTemp", "AvgHumid", "AvgWin"]

# =====================================================
# FUNÇÕES AUXILIARES
# =====================================================

def calcular_importancia_agregada(model, numeric_features, categorical_features):
    rf = model.named_steps["rf"]

    cat_names = []
    try:
        ohe = model.named_steps["preprocess"].named_transformers_["cat"]
        if len(categorical_features) > 0:
            cat_names = list(ohe.get_feature_names_out(categorical_features))
    except Exception:
        cat_names = []

    num_names = list(numeric_features)
    all_feature_names = num_names + cat_names

    importances = pd.Series(rf.feature_importances_, index=all_feature_names)

    agg = {}
    for feat, imp in importances.items():
        if feat in cat_names:
            base = feat.split("_", 1)[0]
        else:
            base = feat
        agg[base] = agg.get(base, 0.0) + imp

    agg_series = pd.Series(agg).sort_values(ascending=False)
    return agg_series


def fazer_grafico_3plots(y_test, y_pred, agg_series, mae, mse, r2, target_col, descricao_dataset, titulo_extra, file_path):
    residuals = y_test - y_pred
    top_10 = agg_series.head(10).sort_values(ascending=True)

    fig, axes = plt.subplots(1, 3, figsize=(25, 8))

    # Plot 1: Real vs Previsto
    ax1 = axes[0]
    sns.scatterplot(x=y_test, y=y_pred, ax=ax1)
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    ax1.plot([min_val, max_val], [min_val, max_val], linestyle="--", color="red")
    ax1.set_xlabel("Casos reais")
    ax1.set_ylabel("Casos previstos")
    ax1.set_title(f"Real vs Previsto — {target_col} {titulo_extra}")
    ax1.text(0.05, 0.95, f"MAE = {mae:.2f}\nMSE = {mse:.2f}\nR² = {r2:.3f}", transform=ax1.transAxes, va="top")

    # Plot 2: Resíduos
    ax2 = axes[1]
    sns.histplot(residuals, bins=20, ax=ax2, kde=True)
    ax2.set_xlabel("Resíduos (Real − Previsto)")
    ax2.set_ylabel("Frequência")
    ax2.set_title("Distribuição dos Resíduos")

    # Plot 3: Importância
    ax3 = axes[2]
    ax3.barh(top_10.index, top_10.values)
    ax3.set_xlabel("Importância (agregada)")
    ax3.set_ylabel("Variáveis")
    ax3.set_title("Top 10 Variáveis Mais Importantes")

    fig.suptitle(f"Random Forest (NoSeason) — {target_col} [{descricao_dataset}] {titulo_extra}", y=1.02, fontsize=14)
    fig.tight_layout()
    fig.savefig(file_path, dpi=300, bbox_inches="tight")
    plt.close(fig)

# =====================================================
# MODELO 1 – RF SPLIT ALEATÓRIO
# =====================================================

def rodar_rf_basico_para_df(df_input, output_folder_path, descricao_dataset):
    df_model = df_input.copy()
    df_model.index = pd.to_datetime(df_model.index)

    # >>> AQUI REMOVEMOS SEASON E SEASON2 <<<
    cols_to_drop = []
    lista_remocao = ["Data", "Ano", "Mes", "MesNum", "Mes_do_ano", "Season", "Season2"]
    
    for c in lista_remocao:
        if c in df_model.columns:
            cols_to_drop.append(c)

    extra_drop = [c for c in df_model.columns if c.endswith("_outlier_IQR") or c.endswith("_extremo")]
    cols_to_drop.extend(extra_drop)

    if cols_to_drop:
        df_model = df_model.drop(columns=cols_to_drop, errors="ignore")

    # Caminho completo incluindo a pasta mãe
    output_dir = base_path / output_folder_path
    output_dir.mkdir(parents=True, exist_ok=True)

    resultados = []

    for target_col in possible_disease_cols:
        if target_col not in df_model.columns:
            continue

        first_idx = df_model[target_col].first_valid_index()
        if first_idx is None:
            continue

        df_target = df_model.loc[first_idx:].copy()
        y = df_target[target_col]
        X = df_target.drop(columns=[c for c in possible_disease_cols if c in df_target.columns], errors="ignore")

        mask_valid = y.notna() & ~X.isna().any(axis=1)
        X = X[mask_valid]
        y = y[mask_valid]

        if len(X) < 10:
            continue

        numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
        categorical_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

        preprocess = ColumnTransformer([
            ("num", "passthrough", numeric_features),
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
        ], remainder="drop")

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        model = Pipeline(steps=[
            ("preprocess", preprocess),
            ("rf", RandomForestRegressor(n_estimators=999, random_state=42, n_jobs=-1))
        ])

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2  = r2_score(y_test, y_pred)

        agg_series = calcular_importancia_agregada(model, numeric_features, categorical_features)

        resultados.append({
            "modelo": "M1_aleatorio_NoSeason",
            "dataset": descricao_dataset,
            "doenca": target_col,
            "MAE": mae, "MSE": mse, "R2": r2
        })

        fazer_grafico_3plots(y_test, y_pred, agg_series, mae, mse, r2, target_col, descricao_dataset, "(split aleatório)", output_dir / f"RF_{target_col}_M1.png")

    return pd.DataFrame(resultados)

# =====================================================
# MODELO 2 – RF COM SPLIT TEMPORAL
# =====================================================

def rodar_rf_temporal_para_df(df_input, output_folder_path, descricao_dataset):
    df_model = df_input.copy()
    df_model.index = pd.to_datetime(df_model.index)
    df_model = df_model.sort_index()

    # >>> AQUI REMOVEMOS SEASON E SEASON2 <<<
    cols_to_drop = []
    lista_remocao = ["Data", "Ano", "Mes", "MesNum", "Mes_do_ano", "Season", "Season2"]
    
    for c in lista_remocao:
        if c in df_model.columns:
            cols_to_drop.append(c)

    extra_drop = [c for c in df_model.columns if c.endswith("_outlier_IQR") or c.endswith("_extremo")]
    cols_to_drop.extend(extra_drop)

    if cols_to_drop:
        df_model = df_model.drop(columns=cols_to_drop, errors="ignore")

    output_dir = base_path / output_folder_path
    output_dir.mkdir(parents=True, exist_ok=True)

    resultados = []

    for target_col in possible_disease_cols:
        if target_col not in df_model.columns: continue
        
        first_idx = df_model[target_col].first_valid_index()
        if first_idx is None: continue

        df_target = df_model.loc[first_idx:].sort_index()
        y = df_target[target_col]
        X = df_target.drop(columns=[c for c in possible_disease_cols if c in df_target.columns], errors="ignore")

        mask_valid = y.notna() & ~X.isna().any(axis=1)
        X = X[mask_valid]
        y = y[mask_valid]

        if len(X) < 10: continue

        split_idx = int(len(X) * 0.75)
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

        numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
        categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

        preprocess = ColumnTransformer([
            ("num", "passthrough", numeric_features),
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
        ], remainder="drop")

        model = Pipeline(steps=[
            ("preprocess", preprocess),
            ("rf", RandomForestRegressor(n_estimators=999, random_state=42, n_jobs=-1))
        ])

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mae, mse, r2 = mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)
        agg_series = calcular_importancia_agregada(model, numeric_features, categorical_features)

        resultados.append({
            "modelo": "M2_temporal_NoSeason",
            "dataset": descricao_dataset,
            "doenca": target_col,
            "MAE": mae, "MSE": mse, "R2": r2
        })

        fazer_grafico_3plots(y_test, y_pred, agg_series, mae, mse, r2, target_col, descricao_dataset, "(split temporal)", output_dir / f"RF_{target_col}_M2.png")

    return pd.DataFrame(resultados)

# =====================================================
# MODELO 3 – RF TEMPORAL + LAGS
# =====================================================

def rodar_rf_temporal_lag_para_df(df_input, output_folder_path, descricao_dataset):
    df_base = df_input.copy()
    df_base.index = pd.to_datetime(df_base.index)
    df_base = df_base.sort_index()

    # >>> AQUI REMOVEMOS SEASON E SEASON2 <<<
    cols_to_drop = []
    lista_remocao = ["Data", "Ano", "Mes", "MesNum", "Mes_do_ano", "Season", "Season2"]
    
    for c in lista_remocao:
        if c in df_base.columns:
            cols_to_drop.append(c)

    extra_drop = [c for c in df_base.columns if c.endswith("_outlier_IQR") or c.endswith("_extremo")]
    cols_to_drop.extend(extra_drop)

    if cols_to_drop:
        df_base = df_base.drop(columns=cols_to_drop, errors="ignore")

    output_dir = base_path / output_folder_path
    output_dir.mkdir(parents=True, exist_ok=True)

    resultados = []

    for target_col in possible_disease_cols:
        if target_col not in df_base.columns: continue
        
        first_idx = df_base[target_col].first_valid_index()
        if first_idx is None: continue

        df_target = df_base.loc[first_idx:].sort_index().copy()
        
        # Criar lags
        df_lag = df_target.copy()
        for lag in [1, 2, 3]:
            for col in climate_cols:
                if col in df_lag.columns:
                    df_lag[f"{col}_lag{lag}"] = df_lag[col].shift(lag)
        
        lag_cols = [c for c in df_lag.columns if "_lag" in c]
        if not lag_cols: continue
        
        df_lag = df_lag.dropna(subset=lag_cols)
        if len(df_lag) < 10: continue

        y = df_lag[target_col]
        X = df_lag.drop(columns=[c for c in possible_disease_cols if c in df_lag.columns], errors="ignore")

        # Limpeza extra de colunas que podem ter sobrado
        for c in lista_remocao:
            if c in X.columns: X = X.drop(columns=[c])

        mask_valid = y.notna() & ~X.isna().any(axis=1)
        X, y = X[mask_valid], y[mask_valid]
        if len(X) < 10: continue

        split_idx = int(len(X) * 0.75)
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

        numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
        categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

        preprocess = ColumnTransformer([
            ("num", "passthrough", numeric_features),
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
        ], remainder="drop")

        model = Pipeline(steps=[
            ("preprocess", preprocess),
            ("rf", RandomForestRegressor(n_estimators=999, random_state=42, n_jobs=-1))
        ])

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mae, mse, r2 = mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)
        agg_series = calcular_importancia_agregada(model, numeric_features, categorical_features)

        resultados.append({
            "modelo": "M3_temporal_lag_NoSeason",
            "dataset": descricao_dataset,
            "doenca": target_col,
            "MAE": mae, "MSE": mse, "R2": r2
        })

        fazer_grafico_3plots(y_test, y_pred, agg_series, mae, mse, r2, target_col, descricao_dataset, "(temporal+lags)", output_dir / f"RF_{target_col}_M3.png")

    return pd.DataFrame(resultados)

# =====================================================
# MODELO 3D – DOMADA
# =====================================================

def rodar_rf_temporal_lag_domada_para_df(df_input, output_folder_path, descricao_dataset):
    df_base = df_input.copy()
    df_base.index = pd.to_datetime(df_base.index)
    df_base = df_base.sort_index()

    # >>> AQUI REMOVEMOS SEASON E SEASON2 <<<
    cols_to_drop = []
    lista_remocao = ["Data", "Ano", "Mes", "MesNum", "Mes_do_ano", "Season", "Season2"]
    
    for c in lista_remocao:
        if c in df_base.columns:
            cols_to_drop.append(c)

    extra_drop = [c for c in df_base.columns if c.endswith("_outlier_IQR") or c.endswith("_extremo")]
    cols_to_drop.extend(extra_drop)

    if cols_to_drop:
        df_base = df_base.drop(columns=cols_to_drop, errors="ignore")

    output_dir = base_path / output_folder_path
    output_dir.mkdir(parents=True, exist_ok=True)

    resultados = []

    for target_col in possible_disease_cols:
        if target_col not in df_base.columns: continue
        
        first_idx = df_base[target_col].first_valid_index()
        if first_idx is None: continue

        df_target = df_base.loc[first_idx:].sort_index().copy()
        
        df_lag = df_target.copy()
        for lag in [1, 2, 3]:
            for col in climate_cols:
                if col in df_lag.columns:
                    df_lag[f"{col}_lag{lag}"] = df_lag[col].shift(lag)
        
        lag_cols = [c for c in df_lag.columns if "_lag" in c]
        if not lag_cols: continue
        
        df_lag = df_lag.dropna(subset=lag_cols)
        if len(df_lag) < 10: continue

        y = df_lag[target_col]
        X = df_lag.drop(columns=[c for c in possible_disease_cols if c in df_lag.columns], errors="ignore")

        for c in lista_remocao:
            if c in X.columns: X = X.drop(columns=[c])

        mask_valid = y.notna() & ~X.isna().any(axis=1)
        X, y = X[mask_valid], y[mask_valid]
        if len(X) < 10: continue

        split_idx = int(len(X) * 0.75)
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

        numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
        categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

        preprocess = ColumnTransformer([
            ("num", "passthrough", numeric_features),
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
        ], remainder="drop")

        model = Pipeline(steps=[
            ("preprocess", preprocess),
            ("rf", RandomForestRegressor(n_estimators=500, max_depth=5, min_samples_leaf=3, max_features="sqrt", random_state=42, n_jobs=-1))
        ])

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mae, mse, r2 = mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)
        agg_series = calcular_importancia_agregada(model, numeric_features, categorical_features)

        resultados.append({
            "modelo": "M3D_domada_NoSeason",
            "dataset": descricao_dataset,
            "doenca": target_col,
            "MAE": mae, "MSE": mse, "R2": r2
        })

        fazer_grafico_3plots(y_test, y_pred, agg_series, mae, mse, r2, target_col, descricao_dataset, "(domada)", output_dir / f"RF_{target_col}_M3D.png")

    return pd.DataFrame(resultados)

# =====================================================
# RODAR TUDO (COM PASTA MÃE E SEM SEASON)
# =====================================================

# Dicionário de bases (Assumindo que os DataFrames já existem na memória)
bases = {
    "base": df,
    "IQR": df_iqr_sem_outliers,
    "p99": df_p99_sem_extremos
}

resultados_list = []

# Caminho relativo para a pasta mãe dentro do base_path
path_mae = Path(pasta_mae_nome)

for desc, df_base_modelo in bases.items():
    print(f"\n>>> Processando Dataset: {desc} (No Season) <<<")
    
    # M1
    p1 = path_mae / f"RF_M1_{desc}_NoSeason"
    resultados_list.append(rodar_rf_basico_para_df(df_base_modelo, p1, desc))
    
    # M2
    p2 = path_mae / f"RF_M2_{desc}_temporal_NoSeason"
    resultados_list.append(rodar_rf_temporal_para_df(df_base_modelo, p2, desc))
    
    # M3
    p3 = path_mae / f"RF_M3_{desc}_lag_NoSeason"
    resultados_list.append(rodar_rf_temporal_lag_para_df(df_base_modelo, p3, desc))
    
    # M3D
    p3d = path_mae / f"RF_M3D_{desc}_domada_NoSeason"
    resultados_list.append(rodar_rf_temporal_lag_domada_para_df(df_base_modelo, p3d, desc))

# Concatenar tudo
resumo_geral = pd.concat(resultados_list, ignore_index=True)

print("\n=== RESUMO GERAL DE TODOS OS MODELOS (NO SEASON) ===")
print(resumo_geral.head())

# =====================================================
# SALVAR EXCEL NA PASTA MÃE
# =====================================================

excel_dir = base_path / pasta_mae_nome
excel_dir.mkdir(parents=True, exist_ok=True)
excel_path = excel_dir / "Tabela_Final_RF_Modelos_NoSeason.xlsx"

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    for doenca in possible_disease_cols:
        df_doenca = resumo_geral[resumo_geral["doenca"] == doenca].copy()
        if df_doenca.empty: continue
        
        # Reordenar colunas se possível
        desired_cols = ["modelo", "dataset", "doenca", "MAE", "MSE", "R2"]
        final_cols = [c for c in desired_cols if c in df_doenca.columns]
        df_doenca = df_doenca[final_cols]

        sheet_name = doenca[:31]
        df_doenca.to_excel(writer, sheet_name=sheet_name, index=False)
    
    resumo_geral.to_excel(writer, sheet_name="ResumoGeral", index=False)

print(f"\nArquivo Excel final salvo em: {excel_path}")


>>> Processando Dataset: base (No Season) <<<


c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating


>>> Processando Dataset: IQR (No Season) <<<


c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating


>>> Processando Dataset: p99 (No Season) <<<


c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
c:\Users\idolo\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating


=== RESUMO GERAL DE TODOS OS MODELOS (NO SEASON) ===
                  modelo dataset    doenca           MAE           MSE  \
0  M1_aleatorio_NoSeason    base    Dengue  22676.013771  1.338383e+09   
1  M1_aleatorio_NoSeason    base      Zika    142.099433  7.661501e+04   
2  M1_aleatorio_NoSeason    base      Chik    651.053637  7.568262e+05   
3  M1_aleatorio_NoSeason    base  LeishVis      3.783511  2.095052e+01   
4  M1_aleatorio_NoSeason    base  LeishTeg      8.275609  1.067565e+02   

         R2  
0 -0.727301  
1 -0.629034  
2 -0.596153  
3  0.042267  
4 -0.326418  

Arquivo Excel final salvo em: C:\Users\idolo\OneDrive\Área de Trabalho\Curso - Modelos Computacionais\Projeto\Resultados_Gerais_NoSeason\Tabela_Final_RF_Modelos_NoSeason.xlsx


## Classificando mês epidêmico vs Não epidêmico 

In [11]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_curve,
)

In [12]:
# ==============================================
# MODELO 4 – CLASSIFICAÇÃO: MÊS EPIDÊMICO vs NÃO (NO SEASON)
# ==============================================

# Caminho base
base_path = Path(r"C:\Users\idolo\OneDrive\Área de Trabalho\Curso - Modelos Computacionais\Projeto")

# Lista de doenças e variáveis climáticas
possible_disease_cols = ["Dengue", "Zika", "Chik", "LeishVis", "LeishTeg", "LeishT"]
climate_cols = ["Precipt", "AvgTemp", "MaxTemp", "MinTemp", "AvgHumid", "AvgWin"]

# Percentil para definir "mês epidêmico"
epidemic_percentile = 0.75  # p75

# Pasta de saída (Atualizada para _NoSeason)
output_dir_class = base_path / "RF_Modelo4_Classificacao_Epidemico_NoSeason"
output_dir_class.mkdir(parents=True, exist_ok=True)


def classificar_meses_epidemicos(
    df_input,
    target_cols=possible_disease_cols,
    percentile_limiar=epidemic_percentile,
    max_lag=3,
    thresholds_list=None,
):
    """
    Roda RandomForestClassifier para prever se um mês é epidêmico (1) ou não (0).
    VERSÃO NO SEASON: Remove explicitamente Season e Season2.
    """

    if thresholds_list is None:
        thresholds_list = [round(t, 2) for t in np.arange(0.1, 1.0, 0.1)]

    # cópia de trabalho
    df_base = df_input.copy()
    df_base.index = pd.to_datetime(df_base.index)
    df_base = df_base.sort_index()

    # >>> REMOÇÃO DE COLUNAS (Incluindo Season/Season2) <<<
    cols_to_drop = []
    lista_remocao = ["Data", "Ano", "Mes", "MesNum", "Mes_do_ano", "Season", "Season2"]
    
    for c in lista_remocao:
        if c in df_base.columns:
            cols_to_drop.append(c)

    # se vierem flags de outlier por engano, também tira
    extra_drop = [c for c in df_base.columns if c.endswith("_outlier_IQR") or c.endswith("_extremo")]
    cols_to_drop.extend(extra_drop)

    if cols_to_drop:
        df_base = df_base.drop(columns=cols_to_drop, errors="ignore")

    resumo_linhas = []
    thresholds_linhas = []

    # Loop por doença
    for target_col in target_cols:
        if target_col not in df_base.columns:
            print(f"[AVISO] {target_col} não está em df. Pulando.")
            continue

        # primeira data com valor não-nulo para essa doença
        first_idx = df_base[target_col].first_valid_index()
        if first_idx is None:
            print(f"[AVISO] {target_col} é toda NaN. Pulando.")
            continue

        print(f"\n=== [Modelo4-NoSeason] Doença alvo: {target_col} ===")
        print(f"Primeira data usada: {first_idx}")

        # Subdataframe a partir do primeiro valor válido
        df_target = df_base.loc[first_idx:].copy()
        df_target = df_target.sort_index()

        # criar mês do ano (1..12) a partir do índice e codificação cíclica
        df_target["Mes_do_ano_temp"] = df_target.index.month
        df_target["Mes_sin"] = np.sin(2 * np.pi * (df_target["Mes_do_ano_temp"] - 1) / 12.0)
        df_target["Mes_cos"] = np.cos(2 * np.pi * (df_target["Mes_do_ano_temp"] - 1) / 12.0)
        df_target = df_target.drop(columns=["Mes_do_ano_temp"])

        # criar lags das variáveis climáticas que existirem
        for lag in range(1, max_lag + 1):
            for col in climate_cols:
                if col in df_target.columns:
                    df_target[f"{col}_lag{lag}"] = df_target[col].shift(lag)

        # variável contínua de casos (para definir o limiar)
        y_cont = df_target[target_col]

        # Definir colunas de X (removendo doenças)
        colunas_possiveis_X = list(df_target.columns)
        for dcol in possible_disease_cols:
            if dcol in colunas_possiveis_X:
                colunas_possiveis_X.remove(dcol)

        X_pre = df_target[colunas_possiveis_X].copy()

        # Remover linhas com NaN em qualquer feature (por causa dos lags)
        mask_valid = (~X_pre.isna().any(axis=1)) & y_cont.notna()
        X_pre = X_pre[mask_valid]
        y_cont_use = y_cont[mask_valid]

        if len(X_pre) < 20:
            print(f"[AVISO] Muito poucos dados válidos para {target_col} após lags/limpeza. Pulando.")
            continue

        # Definir limiar de "mês epidêmico"
        limiar = y_cont_use.quantile(percentile_limiar)
        print(f"Limiar (p{int(percentile_limiar*100)}) para {target_col}: {limiar:.2f} casos")

        # y binário
        y_bin = (y_cont_use >= limiar).astype(int)

        # X definitivo (garantia extra de limpeza)
        X = X_pre.copy()
        for c in lista_remocao: # Garante que Season não sobrou
            if c in X.columns: X = X.drop(columns=[c])
            
        X = X.drop(columns=[c for c in possible_disease_cols if c in X.columns], errors="ignore")

        n_pos = int(y_bin.sum())
        n_total = len(y_bin)
        print(f"{target_col}: {n_pos} meses epidêmicos (1) de {n_total} meses totais ({n_pos/n_total:.1%})")

        # SPLIT TEMPORAL (75% / 25%)
        n = len(X)
        split_idx = int(n * 0.75)

        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y_bin.iloc[:split_idx], y_bin.iloc[split_idx:]

        if y_train.nunique() < 2 or y_test.nunique() < 2:
            print(f"[AVISO] Conjunto de treino ou teste de {target_col} só tem uma classe. Pulando.")
            continue

        # Detectar tipos de colunas
        numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
        categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

        preprocess = ColumnTransformer([
            ("num", "passthrough", numeric_features),
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
        ], remainder="drop")

        # Random Forest Classifier "domada"
        model = Pipeline(steps=[
            ("preprocess", preprocess),
            ("rf", RandomForestClassifier(
                n_estimators=500,
                max_depth=5,
                min_samples_leaf=3,
                max_features="sqrt",
                random_state=42,
                n_jobs=-1
            ))
        ])

        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]

        # AUC
        try:
            auc = roc_auc_score(y_test, y_proba)
        except ValueError:
            auc = np.nan

        # ROC curve plot
        fpr, tpr, roc_thresholds = roc_curve(y_test, y_proba)
        fig_roc, ax_roc = plt.subplots(figsize=(6, 5))
        ax_roc.plot(fpr, tpr, label=f"ROC (AUC = {auc:.3f})")
        ax_roc.plot([0, 1], [0, 1], linestyle="--", label="Aleatório")
        ax_roc.set_xlabel("Falso Positivo (1 - Especificidade)")
        ax_roc.set_ylabel("Verdadeiro Positivo (Sensibilidade)")
        ax_roc.set_title(f"Curva ROC (NoSeason) — {target_col}")
        ax_roc.legend(loc="lower right")
        fig_roc.tight_layout()
        fig_roc.savefig(output_dir_class / f"ROC_{target_col}.png", dpi=300, bbox_inches="tight")
        plt.close(fig_roc)

        # VARREDURA DE THRESHOLDS
        best_thr = None
        best_youden = -np.inf
        best_metrics = None

        for thr in thresholds_list:
            y_pred_thr = (y_proba >= thr).astype(int)
            
            acc = accuracy_score(y_test, y_pred_thr)
            prec = precision_score(y_test, y_pred_thr, zero_division=0)
            rec = recall_score(y_test, y_pred_thr, zero_division=0)
            f1 = f1_score(y_test, y_pred_thr, zero_division=0)
            
            tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thr, labels=[0, 1]).ravel()
            sens = rec
            espec = tn / (tn + fp) if (tn + fp) > 0 else np.nan
            youden = sens + espec - 1 if np.isfinite(sens) and np.isfinite(espec) else -np.inf

            thresholds_linhas.append({
                "doenca": target_col,
                "threshold": thr,
                "AUC": auc,
                "accuracy": acc,
                "precision": prec,
                "recall_sensibilidade": sens,
                "especificidade": espec,
                "f1": f1,
                "youden": youden,
                "n_treino": len(X_train),
                "n_teste": len(X_test)
            })

            if youden > best_youden:
                best_youden = youden
                best_thr = thr
                best_metrics = {
                    "accuracy": acc, "precision": prec, 
                    "recall_sensibilidade": sens, "especificidade": espec, "f1": f1
                }

        if best_thr is None: continue

        print(f"Melhor limiar (Youden): {best_thr:.2f} | Sens: {best_metrics['recall_sensibilidade']:.2f} | Espec: {best_metrics['especificidade']:.2f}")

        # Matriz de confusão no melhor threshold
        y_pred_best = (y_proba >= best_thr).astype(int)
        cm = confusion_matrix(y_test, y_pred_best, labels=[0, 1])
        cm_df = pd.DataFrame(cm, index=["Real 0", "Real 1"], columns=["Prev 0", "Prev 1"])

        fig_cm, ax_cm = plt.subplots(figsize=(6, 5))
        sns.heatmap(cm_df, annot=True, fmt="d", ax=ax_cm)
        ax_cm.set_title(f"Matriz de Confusão (NoSeason) — {target_col}\nThreshold = {best_thr:.2f}")
        fig_cm.tight_layout()
        fig_cm.savefig(output_dir_class / f"CM_{target_col}_best.png", dpi=300, bbox_inches="tight")
        plt.close(fig_cm)

        resumo_linhas.append({
            "doenca": target_col,
            "AUC": auc,
            "best_threshold": best_thr,
            "accuracy_best": best_metrics["accuracy"],
            "precision_best": best_metrics["precision"],
            "recall_sensibilidade_best": best_metrics["recall_sensibilidade"],
            "especificidade_best": best_metrics["especificidade"],
            "f1_best": best_metrics["f1"]
        })

    return pd.DataFrame(resumo_linhas), pd.DataFrame(thresholds_linhas)


# =======================
# Rodar Modelo 4 em df (No Season)
# =======================

# Assumindo que 'df' já existe na memória com os dados originais
resumo_epidemico, thresholds_epidemico = classificar_meses_epidemicos(df)

print("\n=== RESUMO GERAL CLASSIFICAÇÃO EPIDÊMICO vs NÃO (NO SEASON) ===")
print(resumo_epidemico)


# ==============================================
# Salvar tabelas em Excel
# ==============================================

excel_path = output_dir_class / "Tabela_Modelo4_Classificacao_Epidemico_NoSeason.xlsx"

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    if not resumo_epidemico.empty:
        resumo_epidemico.to_excel(writer, sheet_name="Resumo", index=False)

    for doenca in possible_disease_cols:
        df_thr_doenca = thresholds_epidemico[thresholds_epidemico["doenca"] == doenca].copy()
        if df_thr_doenca.empty: continue

        sheet_name = doenca[:31]
        df_thr_doenca.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\nArquivo Excel salvo em: {excel_path}")
print(f"Figuras salvas em: {output_dir_class}")
climate_cols = ["Precipt", "AvgTemp", "MaxTemp", "MinTemp", "AvgHumid", "AvgWin"]  # só usa as que existirem

# Percentil para definir "mês epidêmico"
epidemic_percentile = 0.75  # p75

# Pasta de saída para tudo desse modelo de classificação
output_dir_class = base_path / "RF_Modelo4_Classificacao_Epidemico"
output_dir_class.mkdir(exist_ok=True)


def classificar_meses_epidemicos(
    df_input,
    target_cols=possible_disease_cols,
    percentile_limiar=epidemic_percentile,
    max_lag=3,
    thresholds_list=None,
):
    """
    Roda RandomForestClassifier para prever se um mês é epidêmico (1) ou não (0),
    para cada doença em target_cols, usando:
      - base df_input
      - lags (1..max_lag) das variáveis climáticas disponíveis
      - mês em sin/cos (codificação cíclica)
      - split temporal (75% / 25%)
      - threshold sweep (ex.: 0.1, 0.2, ..., 0.9)
    Retorna:
      - resumo_df: uma linha por doença (AUC, melhor threshold, métricas no melhor threshold)
      - thresholds_all_df: tabela com métricas por doença x threshold
    """

    if thresholds_list is None:
        thresholds_list = [round(t, 2) for t in np.arange(0.1, 1.0, 0.1)]  # 0.1, 0.2, ..., 0.9

    # cópia de trabalho
    df_base = df_input.copy()
    df_base.index = pd.to_datetime(df_base.index)
    df_base = df_base.sort_index()

    # garantir que não tem colunas estranhas de tempo
    cols_to_drop = []
    for c in ["Data", "Ano", "Mes", "MesNum", "Mes_do_ano"]:
        if c in df_base.columns:
            cols_to_drop.append(c)

    # se vierem flags de outlier por engano, também tira
    extra_drop = [c for c in df_base.columns
                  if c.endswith("_outlier_IQR") or c.endswith("_extremo")]
    cols_to_drop.extend(extra_drop)

    if cols_to_drop:
        df_base = df_base.drop(columns=cols_to_drop, errors="ignore")

    resumo_linhas = []
    thresholds_linhas = []

    # Loop por doença
    for target_col in target_cols:
        if target_col not in df_base.columns:
            print(f"[AVISO] {target_col} não está em df. Pulando.")
            continue

        # primeira data com valor não-nulo para essa doença
        first_idx = df_base[target_col].first_valid_index()
        if first_idx is None:
            print(f"[AVISO] {target_col} é toda NaN. Pulando.")
            continue

        print(f"\n=== [Modelo4] Doença alvo: {target_col} ===")
        print(f"Primeira data usada: {first_idx}")

        # Subdataframe a partir do primeiro valor válido
        df_target = df_base.loc[first_idx:].copy()
        df_target = df_target.sort_index()

        # criar mês do ano (1..12) a partir do índice e codificação cíclica
        df_target["Mes_do_ano"] = df_target.index.month
        df_target["Mes_sin"] = np.sin(2 * np.pi * (df_target["Mes_do_ano"] - 1) / 12.0)
        df_target["Mes_cos"] = np.cos(2 * np.pi * (df_target["Mes_do_ano"] - 1) / 12.0)

        # criar lags das variáveis climáticas que existirem
        for lag in range(1, max_lag + 1):
            for col in climate_cols:
                if col in df_target.columns:
                    df_target[f"{col}_lag{lag}"] = df_target[col].shift(lag)

        # variável contínua de casos (para definir o limiar)
        y_cont = df_target[target_col]

        # remover as primeiras linhas que ficaram com NaN nos lags (onde necessário)
        # vamos considerar como features todas as colunas climáticas + seus lags + Mes_sin/cos
        # Primeiro, definimos quais colunas potenciais de X (sem tirar doenças ainda)
        colunas_possiveis_X = list(df_target.columns)

        # mas vamos tirar as próprias doenças (não podem ser preditoras)
        for dcol in possible_disease_cols:
            if dcol in colunas_possiveis_X:
                colunas_possiveis_X.remove(dcol)

        # vamos construir X preliminar
        X_pre = df_target[colunas_possiveis_X].copy()

        # Remover linhas com NaN em qualquer feature (principalmente por causa dos lags)
        # e também garantir que y_cont não seja NaN nessas linhas
        mask_valid = (~X_pre.isna().any(axis=1)) & y_cont.notna()
        X_pre = X_pre[mask_valid]
        y_cont_use = y_cont[mask_valid]

        if len(X_pre) < 20:
            print(f"[AVISO] Muito poucos dados válidos para {target_col} após lags/limpeza. Pulando.")
            continue

        # Definir limiar de "mês epidêmico" (percentil da série dessa doença)
        limiar = y_cont_use.quantile(percentile_limiar)
        print(f"Limiar (p{int(percentile_limiar*100)}) para {target_col}: {limiar:.2f} casos")

        # y binário
        y_bin = (y_cont_use >= limiar).astype(int)

        # Agora, X definitivo: remove qualquer resto de coluna que não queremos
        X = X_pre.copy()

        # garantir que doenças não estão em X (por segurança extra)
        X = X.drop(columns=[c for c in possible_disease_cols if c in X.columns], errors="ignore")

        # tirar Mes_do_ano (já temos sin/cos)
        X = X.drop(columns=["Mes_do_ano"], errors="ignore")

        # conferência de classes
        n_pos = int(y_bin.sum())
        n_total = len(y_bin)
        print(f"{target_col}: {n_pos} meses epidêmicos (1) de {n_total} meses totais "
              f"({n_pos/n_total:.1%} positivos)")

        # SPLIT TEMPORAL (75% / 25%)
        n = len(X)
        split_idx = int(n * 0.75)

        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y_bin.iloc[:split_idx], y_bin.iloc[split_idx:]

        print(f"Treino: {len(X_train)} meses, Teste: {len(X_test)} meses")

        # precisa de pelo menos 1 positivo e 1 negativo em treino e teste
        if y_train.nunique() < 2 or y_test.nunique() < 2:
            print(f"[AVISO] Conjunto de treino ou teste de {target_col} só tem uma classe."
                  f" ROC/AUC ficam indefinidos. Pulando classificação.")
            continue

        # Detectar tipos de colunas
        numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
        categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

        preprocess = ColumnTransformer(
            transformers=[
                ("num", "passthrough", numeric_features),
                ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features),
            ],
            remainder="drop"
        )

        # Random Forest Classifier "domada"
        model = Pipeline(steps=[
            ("preprocess", preprocess),
            ("rf", RandomForestClassifier(
                n_estimators=500,
                max_depth=5,
                min_samples_leaf=3,
                max_features="sqrt",
                random_state=42,
                n_jobs=-1
            ))
        ])

        # treinar e prever
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]

        # AUC geral (independe do threshold)
        try:
            auc = roc_auc_score(y_test, y_proba)
        except ValueError:
            auc = np.nan

        # ROC curve
        fpr, tpr, roc_thresholds = roc_curve(y_test, y_proba)

        # Figura ROC
        fig_roc, ax_roc = plt.subplots(figsize=(6, 5))
        ax_roc.plot(fpr, tpr, label=f"ROC (AUC = {auc:.3f})")
        ax_roc.plot([0, 1], [0, 1], linestyle="--", label="Aleatório")
        ax_roc.set_xlabel("Falso Positivo (1 - Especificidade)")
        ax_roc.set_ylabel("Verdadeiro Positivo (Sensibilidade)")
        ax_roc.set_title(f"Curva ROC — {target_col}")
        ax_roc.legend(loc="lower right")
        fig_roc.tight_layout()

        roc_path = output_dir_class / f"ROC_{target_col}.png"
        fig_roc.savefig(roc_path, dpi=300, bbox_inches="tight")
        plt.close(fig_roc)

        # VARREDURA DE THRESHOLDS (0.1, 0.2, ..., 0.9)
        best_thr = None
        best_youden = -np.inf
        best_metrics = None

        for thr in thresholds_list:
            y_pred_thr = (y_proba >= thr).astype(int)

            acc = accuracy_score(y_test, y_pred_thr)
            prec = precision_score(y_test, y_pred_thr, zero_division=0)
            rec = recall_score(y_test, y_pred_thr, zero_division=0)
            f1 = f1_score(y_test, y_pred_thr, zero_division=0)

            # matriz de confusão pra especificidade
            tn, fp, fn, tp = confusion_matrix(y_test, y_pred_thr, labels=[0, 1]).ravel()
            sens = rec
            espec = tn / (tn + fp) if (tn + fp) > 0 else np.nan

            # Índice de Youden: sens + espec - 1
            youden = sens + espec - 1 if np.isfinite(sens) and np.isfinite(espec) else -np.inf

            thresholds_linhas.append({
                "doenca": target_col,
                "threshold": thr,
                "AUC": auc,
                "accuracy": acc,
                "precision": prec,
                "recall_sensibilidade": sens,
                "especificidade": espec,
                "f1": f1,
                "youden": youden,
                "n_treino": len(X_train),
                "n_teste": len(X_test),
                "n_pos_treino": int(y_train.sum()),
                "n_pos_teste": int(y_test.sum()),
                "limiar_casos_p": percentile_limiar,
                "valor_limiar_casos": limiar
            })

            # Atualizar melhor threshold (maior índice de Youden)
            if youden > best_youden:
                best_youden = youden
                best_thr = thr
                best_metrics = {
                    "accuracy": acc,
                    "precision": prec,
                    "recall_sensibilidade": sens,
                    "especificidade": espec,
                    "f1": f1
                }

        if best_thr is None:
            print(f"[AVISO] Não foi possível definir best_threshold para {target_col}. Pulando resumo.")
            continue

        print(f"Melhor limiar para {target_col} (Youden): {best_thr:.2f}")
        print(f"  Sensibilidade: {best_metrics['recall_sensibilidade']:.3f}  "
              f"Especificidade: {best_metrics['especificidade']:.3f}  "
              f"Acurácia: {best_metrics['accuracy']:.3f}")

        # Matriz de confusão no melhor threshold
        y_pred_best = (y_proba >= best_thr).astype(int)
        cm = confusion_matrix(y_test, y_pred_best, labels=[0, 1])
        cm_df = pd.DataFrame(
            cm,
            index=["Real 0 (não epidêmico)", "Real 1 (epidêmico)"],
            columns=["Previsto 0", "Previsto 1"]
        )

        fig_cm, ax_cm = plt.subplots(figsize=(6, 5))
        sns.heatmap(cm_df, annot=True, fmt="d", ax=ax_cm)
        ax_cm.set_title(f"Matriz de Confusão — {target_col}\nThreshold = {best_thr:.2f}")
        ax_cm.set_ylabel("Real")
        ax_cm.set_xlabel("Previsto")
        fig_cm.tight_layout()

        cm_path = output_dir_class / f"CM_{target_col}_best.png"
        fig_cm.savefig(cm_path, dpi=300, bbox_inches="tight")
        plt.close(fig_cm)

        # Linha do resumo para essa doença
        resumo_linhas.append({
            "doenca": target_col,
            "AUC": auc,
            "best_threshold": best_thr,
            "accuracy_best": best_metrics["accuracy"],
            "precision_best": best_metrics["precision"],
            "recall_sensibilidade_best": best_metrics["recall_sensibilidade"],
            "especificidade_best": best_metrics["especificidade"],
            "f1_best": best_metrics["f1"],
            "n_treino": len(X_train),
            "n_teste": len(X_test),
            "n_pos_treino": int(y_train.sum()),
            "n_pos_teste": int(y_test.sum()),
            "limiar_casos_percentil": percentile_limiar,
            "valor_limiar_casos": limiar
        })

    # Construir DataFrames finais
    resumo_df = pd.DataFrame(resumo_linhas)
    thresholds_all_df = pd.DataFrame(thresholds_linhas)

    return resumo_df, thresholds_all_df


# =======================
# Rodar Modelo 4 em df
# =======================

resumo_epidemico, thresholds_epidemico = classificar_meses_epidemicos(df)

print("\n=== RESUMO GERAL CLASSIFICAÇÃO EPIDÊMICO vs NÃO ===")
print(resumo_epidemico)


# ==============================================
# Salvar tabelas em Excel (uma aba por doença + Resumo)
# ==============================================

tabela_dir = base_path / "RF_Modelo4_Classificacao_Epidemico"
tabela_dir.mkdir(exist_ok=True)

excel_path = tabela_dir / "Tabela_Modelo4_Classificacao_Epidemico.xlsx"

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    # aba Resumo
    resumo_epidemico.to_excel(writer, sheet_name="Resumo", index=False)

    # abas por doença com tabela de thresholds
    for doenca in possible_disease_cols:
        df_thr_doenca = thresholds_epidemico[thresholds_epidemico["doenca"] == doenca].copy()
        if df_thr_doenca.empty:
            continue

        sheet_name = doenca[:31]
        df_thr_doenca.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\nArquivo Excel de classificação salvo em: {excel_path}")
print(f"Figuras (ROC e Matriz de Confusão) salvas em: {output_dir_class}")


=== [Modelo4-NoSeason] Doença alvo: Dengue ===
Primeira data usada: 2014-01-01 00:00:00
Limiar (p75) para Dengue: 33840.00 casos
Dengue: 33 meses epidêmicos (1) de 129 meses totais (25.6%)
Melhor limiar (Youden): 0.10 | Sens: 0.92 | Espec: 0.85

=== [Modelo4-NoSeason] Doença alvo: Zika ===
Primeira data usada: 2016-01-01 00:00:00
Limiar (p75) para Zika: 170.00 casos
Zika: 27 meses epidêmicos (1) de 105 meses totais (25.7%)
Melhor limiar (Youden): 0.30 | Sens: 1.00 | Espec: 0.77

=== [Modelo4-NoSeason] Doença alvo: Chik ===
Primeira data usada: 2017-01-01 00:00:00
Limiar (p75) para Chik: 912.00 casos
Chik: 24 meses epidêmicos (1) de 93 meses totais (25.8%)
Melhor limiar (Youden): 0.20 | Sens: 0.47 | Espec: 1.00

=== [Modelo4-NoSeason] Doença alvo: LeishVis ===
Primeira data usada: 2014-01-01 00:00:00
Limiar (p75) para LeishVis: 14.00 casos
LeishVis: 34 meses epidêmicos (1) de 129 meses totais (26.4%)
Melhor limiar (Youden): 0.30 | Sens: 0.67 | Espec: 0.50

=== [Modelo4-NoSeason] Doença